In [1]:

import torch 
import argparse
import yaml
import time
import multiprocessing as mp
import torch.nn.functional as F
from tabulate import tabulate
from tqdm import tqdm
from torch.utils.data import DataLoader
from pathlib import Path
#from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import GradScaler, autocast
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DistributedSampler, RandomSampler
from torch import distributed as dist
from nmc.models import *
from nmc.datasets import * 
from nmc.augmentations import get_train_augmentation, get_val_augmentation
from nmc.losses import get_loss
from nmc.schedulers import get_scheduler
from nmc.optimizers import get_optimizer
from nmc.utils.utils import fix_seeds, setup_cudnn, cleanup_ddp, setup_ddp
from tools.val import evaluate_epi
from nmc.utils.episodic_utils import * 
from scipy.cluster import hierarchy
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from torchvision import models
import torch.nn as nn
from torch.optim import lr_scheduler
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mutual_info_score
from scipy.cluster import hierarchy
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, hamming_loss
from torch.utils.data import Dataset, DataLoader, Sampler
from torch.utils.data import Subset
import torch.optim as optim
from torchvision import transforms
from PIL import Image
import cv2

In [2]:
with open('../configs/APTOS.yaml') as f:
    cfg = yaml.load(f, Loader=yaml.SafeLoader)
print(cfg)
fix_seeds(3407)
setup_cudnn()
gpu = setup_ddp()
save_dir = Path(cfg['SAVE_DIR'])
save_dir.mkdir(exist_ok=True)
cleanup_ddp()

{'DEVICE': 'cuda:0', 'SAVE_DIR': 'output', 'MODEL': {'NAME': 'EfficientNetV2MModel', 'BACKBONE': 'EfficientNetV2', 'PRETRAINED': '/workspace/jhmoon/nmc_2024/checkpoints/pretrained/tf_efficientnetv2_m_weights.pth', 'UNFREEZE': 'full', 'VERSION': '384_32'}, 'DATASET': {'NAME': 'APTOSDataset', 'ROOT': '/datas/fundus_dataset/APTOS', 'TRAIN_RATIO': 0.7, 'VALID_RATIO': 0.15, 'TEST_RATIO': 0.15}, 'TRAIN': {'IMAGE_SIZE': [384, 384], 'BATCH_SIZE': 32, 'EPOCHS': 100, 'EVAL_INTERVAL': 25, 'AMP': False, 'DDP': False}, 'LOSS': {'NAME': 'CrossEntropy', 'CLS_WEIGHTS': False}, 'OPTIMIZER': {'NAME': 'adamw', 'LR': 0.001, 'WEIGHT_DECAY': 0.01}, 'SCHEDULER': {'NAME': 'warmuppolylr', 'POWER': 0.9, 'WARMUP': 10, 'WARMUP_RATIO': 0.1}, 'EVAL': {'MODEL_PATH': 'checkpoints/pretrained/FGMaxxVit/FGMaxxVit.FGMaxxVit.APTOS.pth', 'IMAGE_SIZE': [384, 384]}, 'TEST': {'MODEL_PATH': 'checkpoints/pretrained/FGMaxxVit/FGMaxxVit.FGMaxxVit.APTOS.pth', 'FILE': 'assests/ade', 'IMAGE_SIZE': [384, 384], 'OVERLAY': True}}


In [3]:
# Early Stopping
class EarlyStopping:
    def __init__(self, patience=7, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_score):
        if self.best_score is None:
            self.best_score = val_score
        elif val_score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = val_score
            self.counter = 0

In [4]:
def get_train_augmentation(size):
    return transforms.Compose([
        transforms.Resize(size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.Lambda(lambda x: x.float() if x.dtype == torch.uint8 else x),
        transforms.Lambda(lambda x: x / 255.0 if x.max() > 1.0 else x),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

def get_val_test_transform(size):
    return transforms.Compose([
        transforms.Resize(size),
        transforms.Lambda(lambda x: x.float() if x.dtype == torch.uint8 else x),
        transforms.Lambda(lambda x: x / 255.0 if x.max() > 1.0 else x),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


In [5]:
class BalancedBatchSampler(Sampler):
    def __init__(self, dataset, batch_size):
        self.dataset = dataset
        self.batch_size = batch_size
        
        # 데이터셋에서 레이블 추출
        if hasattr(dataset, 'labels'):
            self.labels = dataset.labels
            if isinstance(self.labels, np.ndarray):
                self.labels = torch.from_numpy(self.labels)
        elif hasattr(dataset, 'targets'):
            self.labels = dataset.targets
            if isinstance(self.labels, np.ndarray):
                self.labels = torch.from_numpy(self.labels)
        else:
            try:
                self.labels = [sample[1] for sample in dataset]
                if isinstance(self.labels[0], np.ndarray):
                    self.labels = torch.from_numpy(np.array(self.labels))
                else:
                    self.labels = torch.tensor(self.labels)
            except:
                raise ValueError("Cannot access labels from dataset")
        
        self.n_classes = self.labels.shape[1] if len(self.labels.shape) > 1 else len(torch.unique(self.labels))
        self.samples_per_class = batch_size // self.n_classes
        
        # 클래스별 인덱스 저장
        self.class_indices = []
        for i in range(self.n_classes):
            if len(self.labels.shape) > 1:
                idx = torch.where(self.labels[:, i] == 1)[0]
            else:
                idx = torch.where(self.labels == i)[0]
            self.class_indices.append(idx)
        
        self.n_batches = len(self.dataset) // batch_size
        if len(self.dataset) % batch_size != 0:
            self.n_batches += 1
    
    def __iter__(self):
        for _ in range(self.n_batches):
            batch_indices = []
            for class_idx in range(self.n_classes):
                class_samples = self.class_indices[class_idx]
                if len(class_samples) == 0:
                    continue
                
                # 랜덤 선택
                selected = class_samples[torch.randint(len(class_samples), 
                                                     (self.samples_per_class,))]
                batch_indices.extend(selected.tolist())
            
            # 배치 크기에 맞게 자르기
            if len(batch_indices) > self.batch_size:
                batch_indices = batch_indices[:self.batch_size]
            
            # 중요: 리스트로 yield
            yield batch_indices
    
    def __len__(self):
        return self.n_batches

In [6]:
start = time.time()
best_mf1 = 0.0
device = torch.device(cfg['DEVICE'])
print("device : ", device)
num_workers = mp.cpu_count()
train_cfg, eval_cfg = cfg['TRAIN'], cfg['EVAL']
dataset_cfg, model_cfg = cfg['DATASET'], cfg['MODEL']
loss_cfg, optim_cfg, sched_cfg = cfg['LOSS'], cfg['OPTIMIZER'], cfg['SCHEDULER']
epochs, lr = train_cfg['EPOCHS'], optim_cfg['LR']

image_size = [256,256]
image_dir = Path(dataset_cfg['ROOT']) / 'train_images'
train_transform = get_train_augmentation(image_size)
val_test_transform = get_val_test_transform(image_size)
batch_size = 32


dataset = eval(dataset_cfg['NAME'])(
    dataset_cfg['ROOT'] + '/merged_images',
    dataset_cfg['TRAIN_RATIO'],
    dataset_cfg['VALID_RATIO'],
    dataset_cfg['TEST_RATIO'],
    transform=None
)
trainset, valset, testset = dataset.get_splits()
trainset.transform = train_transform
valset.transform = val_test_transform
testset.transform = val_test_transform

# DataLoader 수정
trainloader = DataLoader(
    trainset, 
    batch_sampler=BalancedBatchSampler(trainset, batch_size=batch_size),
    num_workers=num_workers,
    pin_memory=True
)
#trainloader = DataLoader(trainset, batch_size=batch_size, num_workers=num_workers, drop_last=True, pin_memory=True)
valloader = DataLoader(valset, batch_size=1, num_workers=1, pin_memory=True)
testloader = DataLoader(testset, batch_size=1, num_workers=1, pin_memory=True)


device :  cuda:0
/datas/fundus_dataset/APTOS/merged_images
diagnosis
0    1263
2     699
1     259
4     207
3     135
Name: count, dtype: int64
Train size: 2563
diagnosis
0    271
2    150
1     55
4     44
3     29
Name: count, dtype: int64
Validation size: 549
diagnosis
0    271
2    150
1     56
4     44
3     29
Name: count, dtype: int64
Test size: 550


In [8]:
# Model definition (changed to binary classification)
model = models.efficientnet_v2_m(pretrained=True)
num_ftrs = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.BatchNorm1d(num_ftrs),
    nn.Linear(num_ftrs, 5)
)
model = model.to(device)

/root/anaconda3/envs/nmc/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/root/anaconda3/envs/nmc/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=EfficientNet_V2_M_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_M_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
# model for swin S
model = models.swin_s()#pretrained=True)
num_ftrs = model.head.in_features
model.head = nn.Sequential(
    nn.BatchNorm1d(num_ftrs),
    nn.Linear(num_ftrs, 5)
)
model = model.to(device)

In [29]:
# L2 regularization
weight_decay = 1e-4
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001, weight_decay=weight_decay)
criterion = nn.BCEWithLogitsLoss()
scaler = GradScaler(enabled=train_cfg['AMP'])
# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5, verbose=True)


In [30]:

def train_epoch(model, dataloader, criterion, optimizer, scaler, device):
    model.train()
    total_loss = 0
    for images, labels in tqdm(dataloader, desc="Training"):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        with autocast(enabled=scaler is not None):
            outputs = model(images)
            loss = criterion(outputs.squeeze(), labels.float())
        
        if scaler is not None:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

In [31]:
def evaluate(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            # 각 클래스에 대해 시그모이드 적용 후 임계값 처리
            preds = (torch.sigmoid(outputs) > 0.5).int()
            
            # 배치 단위로 예측값과 라벨 저장
            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
    
    # 배치 데이터를 하나의 배열로 결합
    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)
    
    # 멀티라벨 F1 score 계산
    f1 = f1_score(all_labels, all_preds, average='samples')  # or 'micro', 'macro', 'weighted'
    
    return f1

In [32]:
def train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, scaler, device, epochs):
    best_f1 = 0.0
    early_stopping = EarlyStopping(patience=10, min_delta=0.001)
    
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        
        train_loss = train_epoch(model, train_loader, criterion, optimizer, scaler, device)
        val_f1 = evaluate(model, val_loader, device)
        
        print(f"Training Loss: {train_loss:.4f}")
        print(f"Validation F1 Score: {val_f1:.4f}")
        
        scheduler.step(val_f1)
        
        if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save(model.state_dict(), 'model/best_model_aptos.pth')
            print("New best model saved!")
        
        early_stopping(val_f1)
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break
        
        print()
    
    return best_f1

In [33]:
# Main execution code
# 정규화, lr스케쥴링, 데이터 증강, 조기종료, 배치정규화
epochs = 100
best_f1 = train_and_evaluate(model, trainloader, valloader, criterion, optimizer, scaler, device, epochs)

print(f"Training completed. Best F1 Score: {best_f1:.4f}")

# Final evaluation on test set
model.load_state_dict(torch.load('model/best_model_aptos.pth'))
test_f1 = evaluate(model, testloader, device)
print(f"Test F1 Score: {test_f1:.4f}")

Epoch 1/100


Evaluating: 100%|██████████| 549/549 [01:05<00:00,  8.36it/s]


Training Loss: 0.5606
Validation F1 Score: 0.7208
New best model saved!

Epoch 2/100


Evaluating: 100%|██████████| 549/549 [01:10<00:00,  7.81it/s]


Training Loss: 0.3896
Validation F1 Score: 0.7571
New best model saved!

Epoch 3/100


...

Training Loss: 0.0138
Validation F1 Score: 0.8349

Epoch 32/100


Evaluating: 100%|██████████| 549/549 [01:13<00:00,  7.51it/s]


Training Loss: 0.0116
Validation F1 Score: 0.8379
Early stopping triggered
Training completed. Best F1 Score: 0.8488


Evaluating: 100%|██████████| 550/550 [01:30<00:00,  6.07it/s]

Test F1 Score: 0.8339


In [8]:
def evaluate(model, dataloader, device, num_classes):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            # 각 클래스에 대해 시그모이드 적용 후 임계값 처리
            preds = (torch.sigmoid(outputs) > 0.5).int()
            
            # 배치 단위로 예측값과 라벨 저장
            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
    
    # 배치 데이터를 하나의 배열로 결합
    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)
    
    # 전체 F1 score 계산
    overall_f1 = f1_score(all_labels, all_preds, average='samples')
    
    # 각 클래스별 F1 score 계산
    class_f1_scores = f1_score(all_labels, all_preds, average=None)
    
    # 각 클래스별 정밀도(Precision)와 재현율(Recall) 계산
    class_precision = precision_score(all_labels, all_preds, average=None)
    class_recall = recall_score(all_labels, all_preds, average=None)
    
    # 결과를 딕셔너리로 정리
    results = {
        'overall_f1': overall_f1,
        'class_f1_scores': class_f1_scores,
        'class_precision': class_precision,
        'class_recall': class_recall
    }
    
    # 각 클래스별 메트릭 출력
    print("\nPer-class Performance Metrics:")
    print("-" * 50)
    for i in range(num_classes):
        print(f"Class {i}:")
        print(f"  F1-Score: {class_f1_scores[i]:.4f}")
        print(f"  Precision: {class_precision[i]:.4f}")
        print(f"  Recall: {class_recall[i]:.4f}")
    print("-" * 50)
    print(f"Overall F1-Score: {overall_f1:.4f}")
    
    return results

In [ ]:
model.load_state_dict(torch.load('model/best_model_aptos.pth'))
test_f1 = evaluate(model, testloader, device, 5)

Evaluating:  48%|████▊     | 266/550 [00:44<00:28,  9.88it/s]

Evaluating: 100%|██████████| 550/550 [01:36<00:00,  5.68it/s]


Per-class Performance Metrics:
--------------------------------------------------
Class 0:
  F1-Score: 0.9832
  Precision: 0.9925
  Recall: 0.9742
Class 1:
  F1-Score: 0.5155
  Precision: 0.6098
  Recall: 0.4464
Class 2:
  F1-Score: 0.7943
  Precision: 0.6950
  Recall: 0.9267
Class 3:
  F1-Score: 0.2791
  Precision: 0.4286
  Recall: 0.2069
Class 4:
  F1-Score: 0.7013
  Precision: 0.8182
  Recall: 0.6136
--------------------------------------------------
Overall F1-Score: 0.8339


In [9]:
model.load_state_dict(torch.load('model/multilabel/best_model_aptos_vit.pth'))
#model = torch.load('model/multilabel/best_model_aptos_cnn.pth', map_location=device)
test_f1 = evaluate(model, testloader, device, 5)

Evaluating: 100%|██████████| 550/550 [01:04<00:00,  8.55it/s]


Per-class Performance Metrics:
--------------------------------------------------
Class 0:
  F1-Score: 0.9796
  Precision: 0.9851
  Recall: 0.9742
Class 1:
  F1-Score: 0.5263
  Precision: 0.3913
  Recall: 0.8036
Class 2:
  F1-Score: 0.7658
  Precision: 0.6526
  Recall: 0.9267
Class 3:
  F1-Score: 0.4471
  Precision: 0.3393
  Recall: 0.6552
Class 4:
  F1-Score: 0.5714
  Precision: 0.6000
  Recall: 0.5455
--------------------------------------------------
Overall F1-Score: 0.8155
